In [43]:
import time, random, pickle
from collections import defaultdict, deque
from tqdm import tqdm

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

from pyvirtualdisplay import Display
from webdriver_manager.chrome import ChromeDriverManager



In [44]:
from pyvirtualdisplay import Display


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


path='chromedriver/chromedriver'

driver = webdriver.Chrome(executable_path=path, options=chrome_options)

In [45]:
pickle_data_path = './data/image0_sort_popular_overall_asin.pickle'

with open(pickle_data_path, "rb") as f:
    asin_number = pickle.load(f)



In [50]:
start_url = 'https://www.amazon.com/'

driver.get(url=start_url)

informations = defaultdict(list)

# 0002103818 no results
for num in tqdm(range(93, len(asin_number))):
    if num % 1000 == 0:
        with open('full_image_url.pickle', 'ab') as f:
            pickle.dump(informations, f)
            informations = defaultdict(list)
        print('by ' + str(num) + ', completed')

    number = asin_number[num]
    
    assignment = True
    sleep_k = 0

    while assignment:
        if driver.title == "Sorry! Something went wrong!":
            print(str(sleep_k) + ' step slept..',end='\r', flush=True)
            time.sleep(10)

            driver.close()
            driver = webdriver.Chrome(executable_path=path, options=chrome_options)
            driver.get(url=start_url)
            sleep_k += 1

        else:
            search_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID,"twotabsearchtextbox"))
            )
            search_box.clear()

            search_box.send_keys(number)
            search_box.send_keys(Keys.RETURN)

            assignment = False

    time.sleep(2*random.random())

    items_title_info = driver.find_elements(by=By.CLASS_NAME, value='s-title-instructions-style')
    items_image_info = driver.find_elements(by=By.CLASS_NAME, value='s-image')

    for i in range(len(items_title_info)):
        item_title = None
        item_img_link = None

        item_title_info = items_title_info[i]
        item_image_info = items_image_info[i]
        
        try:
            item_title_text = item_title_info.text.split('\n')
            item_title = item_title_text[0]
            item_img_link = item_image_info.get_attribute('src')

            informations[number].append(item_title)
            informations[number].append(item_img_link)
        
        except:
            print('error_number:', number)
            continue



  0%|          | 2/418772 [00:06<396:27:13,  3.41s/it]


KeyboardInterrupt: 

In [51]:
print(informations)

defaultdict(<class 'list'>, {'1547042389': ['Billionaire In Vegas (Billionaire Matchmaker Book 1)', 'https://m.media-amazon.com/images/I/81NV-A3P3+L._AC_UY218_.jpg']})


In [11]:
driver.close()

81

<class 'collections.defaultdict'>
81
0996135693
1979980659
1974123650
1979519153
1983500933
1978393296
1682304302
B001C4VLZQ
B003156C4E
1519083831
B00A9WRH3M


In [21]:
with open('full_image_url.pickle', 'wb') as f:
    pickle.dump(informations, f)